# Set the file paths
In the cell with TSV_FILE_PATH set the path to tsv file and the path to the files corresponding to the correct experiment number. Only the prefix is required and not the full file names. Afterwards run the cell to set the variables (Press play button while selected).

**NOTE:** the paths should either be absolute paths (c:/Users/enzomo/Downloads/enzo_data/...) or from the current directory. Execute the cell below to see the current directory (can use this to creat absolute paths if needed). Alternatively, on windows can right-click on file and select "copy as path"

In [ ]:
import os

os.getcwd()

In [ ]:
# FILL in path to tsv file
TSV_FILE_PATH="/Users/patbu/Desktop/ecms_np_analysis/example/20240905_VSP_Au_4nm_2_5min_1M_KHCO3/2024-09-10 16_49_03 Full Test/2024-09-10 16_49_03 Full Test.tsv"

# FILL in path to experiment number files
EXPT_NUM_FILE_PATH="/Users/patbu/Desktop/ecms_np_analysis/example/20240905_VSP_Au_4nm_2_5min_1M_KHCO3/03__02_CP"

In [ ]:
mass_to_species = {
    "M2": "H$_{2}$",
    "M4": "He",
    "M15": "CH$_{4}$",
    "M18": "H$_{2}$O",
    "M26": "C$_{2}$H$_{2}$",
    "M28": "CO/N$_{2}$",
    "M32": "O$_{2}$",
    "M40": "Ar",
    "M44": "CO$_{2}$",
}
TSV_FILENAME = os.path.basename(TSV_FILE_PATH)[:-4]
TSV_FILE_DIR = os.path.dirname(TSV_FILE_PATH)

In [ ]:
from ixdat import Measurement


ms = Measurement.read(
    TSV_FILE_PATH,
    reader="zilien",
    technique="MS",
)

# ms.plot()

ec = Measurement.read_set(
    EXPT_NUM_FILE_PATH,
    suffix=".mpt",
)

ecms = ec + ms
# ecms["raw_potential"]._data = ecms["<Ewe/V>"]._data # fix for raw_potential being always 0 (not read correctly) 
ecms["raw_potential"]._data = ecms["<Ewe/V>"]._data


In [ ]:
# reset the time origin to zero if needed
ecms.t[0]
ecms.tstamp += ecms.t[0] - 1

# Plot ecms
can save/copy plot by hovering over it 

In [ ]:
axes = ecms.plot()
fig = axes[0].get_figure()
fig.set_size_inches(
    7, # figure width
    8, # figure height
    # forward=True
)

handles, labels = axes[0].get_legend_handles_labels()
new_labels = [ mass_to_species[x] for x in labels]
new_labels

axes[0].legend(
    loc=(1.05, 0.5), # uncomment to change legend position
    labels=new_labels,
)



fig.suptitle(TSV_FILENAME)

Alternatively uncomment the following to save in same folder as tsv file

In [ ]:
# fig.savefig(os.path.join(TSV_FILE_DIR, "ecms_plot.png"), dpi=300, bbox_inches='tight',)

# FILL IN: Parameters for Calculating Faradaic Efficiency
set the parameters required for processing the data below. Importantly require the step length for each applied current and the steps (starting from step 0) to calculate faradaic efficiency of CO2RR. Also, provide either the calibration factor and MS background current for H2 reduction (HER) or else the steps/times that should be used to calculate it. Optionally provide the start time (if not 0) and the time prior to end of step to average over for calculations.

In [ ]:
STEP_TIME = 300 # seconds

# for calculating the calibration factor from MS current to raw/cell current
# fill in one of the following
HER_ONLY_STEPS = [3, 4, 5,] # step numbers in sequence/program, first step is 1
HER_CALIBRATION_FACTOR = None

# fill in one of the following
HER_BACKGROUND_TIME_INTERVAL = [5100, 5200] # for calculating the background current of the ms
HER_BACKGROUND_CURRENT = None

TIME_AMOUNT_AVERAGE_OVER = 100 # seconds
START_TIME = 0 # if want to start steps from a time other that t=0


# parameter check
assert HER_ONLY_STEPS or HER_CALIBRATION_FACTOR is not None, "must provide HER calibration factor OR the step numbers for calculating it"
assert HER_BACKGROUND_TIME_INTERVAL or HER_BACKGROUND_CURRENT is not None, "must provide H2 MS background current OR time interval to calculate it"

### Optional: Add electrode details

In [ ]:
# ecms["raw_potential"]._data = meas["<Ewe/V>"]._data
ecms.calibrate( # do this first
    RE_vs_RHE=0, # if use RHE reference electrode assume that the potential difference between our reference electrode and the RHE potential is zero.
    A_el=0.196, # We know the geometric area of the electrode, so we can normalize the current: it’s a 5mm diameter disk, area = 0.196 cm^2
    R_Ohm=0, # We did not determine the Ohmic drop, but we will assume that it was 0 
)

### Calculate HER background current if required

In [ ]:
if HER_BACKGROUND_CURRENT is None:
    times, currents = ecms.grab(item="M2 [A]", tspan=HER_BACKGROUND_TIME_INTERVAL)
    # M2_ave_current = M2_current.mean()
    HER_BACKGROUND_CURRENT = currents.mean()

print("HER MS background current:", HER_BACKGROUND_CURRENT)

### Calculate HER calibration factor if required using HER only steps

In [ ]:
if HER_CALIBRATION_FACTOR is None:
    
    axes = ecms.plot()
    cal_result_H2, ax_b2 = ecms.ecms_calibration_curve(
        mol="H2",
        mass="M2",
        n_el=-2, # remember to use the correct sign: minus for reduction reactions, plus for oxidation reactions
        tspan_list=[
            (START_TIME + STEP_TIME*(x)-TIME_AMOUNT_AVERAGE_OVER, START_TIME + STEP_TIME*(x))
            for x in HER_ONLY_STEPS
        ],
        ax="new",
        axes_measurement=axes, # to highlight the integrated areas on the plot defined above
        return_ax = True # if True, returns the calibration curve axis as a second element
    )
    print(cal_result_H2) 
    HER_CALIBRATION_FACTOR=cal_result_H2.F # The attribute cal_result_H2.F is the slope of the calibration curve, which is the sensitivity factor in C/mol. 



## Fit conversion between MS current and Cell current using HER only steps

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

HER_calibration_intervals = [
    (START_TIME + STEP_TIME*(x)-TIME_AMOUNT_AVERAGE_OVER, START_TIME + STEP_TIME*(x))
    for x in HER_ONLY_STEPS
]

raw_currents, HER_MS_currents = [], []

for interval in HER_calibration_intervals:
    raw_currents.append(
        ecms.grab(item="raw_current", tspan=interval)[1].mean(),
    )
    HER_MS_currents.append(
        ecms.grab(item="M2 [A]", tspan=interval)[1].mean(),
    )

raw_currents = np.array(raw_currents)
HER_MS_currents = np.array(HER_MS_currents) - HER_BACKGROUND_CURRENT

plt.scatter(
    HER_MS_currents,
    raw_currents,
)

m, b = np.polyfit(
    HER_MS_currents,
    raw_currents,
    1
)

plt.plot(
    HER_MS_currents,
    [m*v+b for v in HER_MS_currents]
)

print("y=mx+b fit coefs:", m, b)

# Calculate CO2RR Faradaic Efficiencies

In [ ]:
FE_data = []

all_intervals = [
    (START_TIME + STEP_TIME*(x+1)-TIME_AMOUNT_AVERAGE_OVER, START_TIME + STEP_TIME*(x+1))
    for x in range(ecms.selector._data[-1] + 1)
]

for interval in all_intervals:
    raw_current = ecms.grab(item="raw_current", tspan=interval)[1].mean()
    HER_MS_current = ecms.grab(item="M2 [A]", tspan=interval)[1].mean() - HER_BACKGROUND_CURRENT

    HER_cell_current = HER_MS_current * m + b
    CO2RR_cell_current = raw_current - HER_cell_current

    # don't calculate Faradaic efficiency if current is close to background
    if (
        HER_MS_current < HER_BACKGROUND_CURRENT or
        np.isclose(HER_MS_current, HER_BACKGROUND_CURRENT, rtol=0.10, atol=1e-14)
    ):
        CO2RR_cell_current = 0.00
        faradaic_efficiency = 0.0
    else:
        faradaic_efficiency = (CO2RR_cell_current / raw_current) * 100

    FE_data.append([
        interval[0], interval[1],
        raw_current,
        HER_MS_current,
        HER_cell_current,
        CO2RR_cell_current,
        faradaic_efficiency
    ])

FE_data = pd.DataFrame(
    FE_data,
    columns=[
        "start (s)", "end (s)",
        "total cell current (A)",
        "HER MS current (A)",
        "HER cell current (A)",
        "CO2RR cell current (A)",
        "Faradaic Efficiency (%)"]
)

In [ ]:
FE_data

uncomment following line to save csv file in same folder as tsv file

In [ ]:
# FE_data.to_csv(os.path.join(TSV_FILE_DIR, "CO2RR_faradaic_efficiencies.csv"), index=False)

### MS currents csv file

In [ ]:
mass_list = sorted(ecms.mass_list, key=lambda x: float(x[1:]))

mass_to_species = {
    "M2": "H2",
    "M4": "He",
    "M15": "CH4",
    "M18": "H2O",
    "M26": "C2H2",
    "M28": "CO/N2",
    "M32": "O2",
    "M40": "Ar",
    "M44": "CO2",
}

all_intervals = [
    (START_TIME + STEP_TIME*(x+1)-TIME_AMOUNT_AVERAGE_OVER, START_TIME + STEP_TIME*(x+1))
    for x in range(ecms.selector._data[-1] + 1)
]


all_currents = []
for i, interval in enumerate(all_intervals):
    interval_currents = [i, *interval]
    for mass in mass_list:
        interval_currents.append(ecms.grab(f"{mass} [A]", tspan=interval)[1].mean())

    all_currents.append(interval_currents)


all_currents

column_names = ["step num", "start time", "end time",] + [f"{mass_to_species[x]} ave MS current [A]" for x in mass_list]

ms_currents_data = pd.DataFrame(all_currents, columns=column_names)

In [ ]:
ms_currents_data

uncomment the following to save data in same folder as tsv file

In [ ]:
# ms_currents_data.to_csv(os.path.join(TSV_FILE_DIR, "all_ms_currents.csv"), index=False)